In [2]:
import sys
!{sys.executable} -m pip install pandas


In [4]:
import pandas as pd
import os
import re

In [20]:
file_name = 'localapp/data/raw_messages.csv'
df = pd.read_csv(file_name)


In [ ]:
#get correct pattern to test against
# pat = '^[a-zA-Z0-9,.]+$'
# string = 'A,5$1.81%8691333&3333@34*,N,5.3762513@33333333'
# print(bool(re.match(pat, string)))
# s = re.sub('[^0-9a-zA-Z,.]+', '', string)


In [21]:
# clean raw_message & transform date
clean_msg = pd.Series(df.raw_message).str.replace('[^0-9a-zA-Z,.]+', '', regex=True)
df["clean_msg"] = clean_msg


In [22]:
def convert_columns(df):
    df["datetime"] = df.datetime.apply(lambda x: pd.to_datetime(x, unit = 's'))
    df = df.astype(dtype= {
    "lat":"float64", 
    "lon":"float64", 
    "spd_over_grnd":"float64", 
    "true_course":"float64", 
    "mag_variation":"float64",
    "datestamp":"int64"})
    return df


In [23]:
msg_content_header = ["status", "lat", "lat_dir", "lon", "lon_dir", "spd_over_grnd", "true_course", "datestamp", "mag_variation", "mag_var_dir"]
msg_field = "clean_msg"

def split_string_column(input_df, msg_field, msg_content_header):
    col_df = pd.DataFrame(input_df[msg_field].str.split(pat=',', expand=True))
    col_df.columns = msg_content_header
    return col_df

df_message = split_string_column(df, msg_field, msg_content_header)
df_split = df.join(df_message)

df_split.drop(['raw_message', 'clean_msg'], axis=1, inplace=True)
df_split = convert_columns(df_split)



In [24]:
df_split.head()

,device_id,datetime,address_ip,address_port,original_message_id,status,lat,lat_dir,lon,lon_dir,spd_over_grnd,true_course,datestamp,mag_variation,mag_var_dir
0,0001,2019-02-13 14:09:59,172.19.0.17,4007,1550070599576-0,A,51.318308,N,4.315722,E,0.0,1.59,150218,0.8,E
1,0001,2019-02-13 14:21:01,172.19.0.16,4007,1550071261429-0,A,51.318308,N,4.315722,E,0.0,1.59,150218,0.8,E
2,0001,2019-02-13 14:10:48,172.19.0.17,4007,1550070648974-0,A,51.318309,N,4.315721,E,0.0,5.25,150218,0.8,E
3,0001,2019-02-13 14:43:54,172.19.0.16,4007,1550072634730-0,A,51.318309,N,4.315721,E,0.0,5.25,150218,0.8,E
4,0001,2019-02-13 14:54:56,172.19.0.16,4007,1550073296534-0,A,51.318309,N,4.315721,E,0.0,5.25,150218,0.8,E


In [25]:
# Check direction fields for errors. Any column that contains more than 2 distinct values contains extra characters.
print(df_split.status.unique().size, 
df_split.lat_dir.unique().size,
df_split.lon_dir.unique().size,
df_split.mag_var_dir.unique().size)



2 1 1 1


In [26]:
df_provided = pd.read_csv('localapp/data/raw_messages_clean.csv')
df_provided.head()

df_provided.head()

,device_id,datetime,address_ip,address_port,original_message_id,status,lat,lat_dir,lon,lon_dir,spd_over_grnd,true_course,datestamp,mag_variation,mag_var_dir
0,0001,2019-02-13 14:09:59,172.19.0.17,4007,1550070599576-0,A,51.318308,N,4.315722,E,0.0,1.59,150218,0.8,E
1,0001,2019-02-13 14:21:01,172.19.0.16,4007,1550071261429-0,A,51.318308,N,4.315722,E,0.0,1.59,150218,0.8,E
2,0001,2019-02-13 14:10:48,172.19.0.17,4007,1550070648974-0,A,51.318309,N,4.315721,E,0.0,5.25,150218,0.8,E
3,0001,2019-02-13 14:43:54,172.19.0.16,4007,1550072634730-0,A,51.318309,N,4.315721,E,0.0,5.25,150218,0.8,E
4,0001,2019-02-13 14:54:56,172.19.0.16,4007,1550073296534-0,A,51.318309,N,4.315721,E,0.0,5.25,150218,0.8,E


In [29]:
df_provided.compare(df_split)

lat             lon      
            self      other self other
37     51.901005  51.901005  NaN   NaN
38     51.901005  51.901005  NaN   NaN
39     51.901005  51.901005  NaN   NaN
40     51.901005  51.901005  NaN   NaN
41     51.901005  51.901005  NaN   NaN
...          ...        ...  ...   ...
29000  51.818092  51.818092  NaN   NaN
29004  51.819714  51.819714  NaN   NaN
29034  51.818930  51.818930  NaN   NaN
29038  51.818835  51.818835  NaN   NaN
29048  51.818691  51.818691  NaN   NaN

[2712 rows x 4 columns]